ПАРСЕР

Duma

В этом примере мы зайдем на сайт государственной думы и попробуем получить данные обо всех ее депутатах, в какой фракции и комитете он состоит, а также занимаю должность. Все это в итоге представим в виде DataFrame и сохраним как csv файл, для дальнейшей работы.

Делаем все как обычно, заходим на сайт думы, используем requests и BeatifulSoup

In [11]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

url = "http://duma.gov.ru/duma/deputies/7/"
page = requests.get(url)

text = page.text
soup = BeautifulSoup(text, "html.parser")

Имена депутатов будет хранить в deputates, комитеты в которых они состоят в commissions, должность в occupations и партию в party.

In [12]:
deputates = []
commissions = []
occupations = []
party = []

Депутатов много (487 человек) и скрейпинг может занимать прилично времени. Поэтому ограничим количество обработанных человек с помощью счетчика count и проверки. Для примера оставим 10 человек.

Данные обо всхе депутатах находятся в теге "a" и с классом "personimage-wrapper personimage-wrapper--s"

Найдем все такие теги

In [3]:
count = 0    
person = soup.find_all('a', class_ = "person__image-wrapper person__image-wrapper--s")

In [4]:
person[0]

<a class="person__image-wrapper person__image-wrapper--s" href="/duma/persons/99112936/" itemprop="url">
<picture>
<img alt="Авдеев Александр Александрович" class="person__image person__image--s" itemprop="image" src="/media/persons/240x240/OM9mmw6lAZ1lhnpM1yYBIMAo1BaQgxnz.jpg"/>
</picture>
</a>

In [5]:

for p in person:
    print(p.find('img'))
    name = p.find('img')
    print(name.get('alt'))
    print(p.get('href'))
    print(deputates)
    url_deput = "http://duma.gov.ru/" + p.get("href")
    print(url_deput)
    page = requests.get(url_deput)
    soup_deput = BeautifulSoup(page.text, "html.parser")
    comm = soup_deput.find('a', class_ = "link link--underline link--external")
    #print(comm)
    t = comm.text.strip()
    t_new = t[:t.find('(')]
    print(t_new)
    break

<img alt="Авдеев Александр Александрович" class="person__image person__image--s" itemprop="image" src="/media/persons/240x240/OM9mmw6lAZ1lhnpM1yYBIMAo1BaQgxnz.jpg"/>
Авдеев Александр Александрович
/duma/persons/99112936/
[]
http://duma.gov.ru//duma/persons/99112936/
Комитет по контролю и Регламенту 


In [6]:
t = comm.text.strip()
t[t.find('(')]

'('

In [7]:
for p in person:  
    
    # Обрезаем скрейпинг, т.к. очень долго идет обработка
    if count > 10:
        break
    
    #Получаем имена
    name = p.find('img', class_ = 'person__image person__image--s') # находим тег img и забираем из него значение alt
    deputates.append(name.get("alt"))
    
    # Получаем ссылку на депутата
    url_deput = "http://duma.gov.ru/" + p.get("href") # в нашем найденном теге уже есть нужная ссылка на депутата. Забираем ее
    
    # Делаем запрос
    page = requests.get(url_deput)
    soup_deput = BeautifulSoup(page.text, "html.parser")
    
    #Ищем комитет
    comm = soup_deput.find('a', class_ = "link link--underline link--external")
    t = comm.text.strip("\n")
    # регулярным выражением убираем все непечатный символы
    commissions.append(re.sub('\W+',' ',t[:t.index("(")-1]).strip()) # убираем должность
    occupations.append(t[t.index("(") + 1:t.index(")")]) #должность всегда в скобочках, 
                                                         #находим первые вхождени ( и )

    # Поиск фракции
    part = soup_deput.find('a', class_ = "link link--underline person__description__link")
    party.append(part.text)

    count += 1 # обновляем счетчик

In [8]:
df = pd.DataFrame({'Депутат': deputates, 'Коммитет':commissions ,'Должность':occupations, 'Фракция': party})
df.to_csv("Gos_duma.csv")

In [9]:
df.head()

,Депутат,Коммитет,Должность,Фракция
0,Авдеев Александр Александрович,Комитет по контролю и Регламенту,член комитета,«ЕДИНАЯ РОССИЯ»
1,Авдеев Михаил Юрьевич,Комитет по транспорту и строительству,Первый заместитель председателя комитета,КПРФ
2,Агаев Ваха Абуевич,Комитет по финансовому рынку,заместитель председателя комитета,КПРФ
3,Адучиев Батор Канурович,Комитет по аграрным вопросам,член комитета,«ЕДИНАЯ РОССИЯ»
4,Азимов Рахим Азизбоевич,Комитет по безопасности и противодействию корр...,член комитета,«ЕДИНАЯ РОССИЯ»
